In [1]:
from bs4 import BeautifulSoup
import httplib
import urllib2
import pandas as pd
import numpy as np

from datetime import date, timedelta, datetime
from time import sleep
import os.path
import re
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
if not os.path.exists('datasets/s_p_500.csv'):
    s_p_500 = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies', attrs = {"class":'wikitable sortable'}, header = 0)[0]
    s_p_500.to_csv('datasets/s_p_500.csv', encoding = "utf-8")
else:
    s_p_500 = pd.read_csv('datasets/s_p_500.csv', index_col = 0)

s_p_500.columns = [x.strip().replace(' ', '_') for x in s_p_500.columns]
s_p_500.Ticker_symbol = s_p_500.Ticker_symbol.apply(lambda x: x.replace('-', ''))
s_p_500['Ticker_symbol'] = s_p_500['Ticker_symbol'].apply(lambda x: x.replace('.N', ''))

In [3]:
def get_article(href, text, date):
    link = {}
    link['Url'] = "http://www.reuters.com/"+href
    link['Title'] = text
    link['Symbol'] = symbol
    link['Date'] = date
    soup = BeautifulSoup(urllib2.urlopen(link['Url']), "lxml")
    link['Article'] = soup.find_all(
        'span', {"id":"article-text"})[0].text.replace('\n', ' ')
    link['Time'] = soup.find_all(
        'span', {"class":"timestamp"})[0].text
    sleep(random.randint(1,5)/10.0)
    return link

def symbol_news(url):
    links=[]
    news = BeautifulSoup(urllib2.urlopen(url), "lxml").find_all('div', {"id":"companyNews"})
    for i in xrange(2):
#         print url
        for feature in news[i].find_all('h2'):
            a = feature.find('a')
            if a.has_attr('href'):
                d = re.search('[0-9]{8}', url).group(0)
                day=datetime.strptime(d, '%m%d%Y').date()
                links.append(get_article(a['href'], a.text, day))
    sleep(random.randint(1,5)/10.0)
    return links

def get_news_for_symbol(symbol, start_date = date(2014, 1, 1), end_date = date(2014, 2, 1)):
    days = [start_date + timedelta(n) for n in range((end_date - start_date).days)]
    links = []
    for day in days:
        url = "http://www.reuters.com/finance/stocks/companyNews?symbol={}&date={}".format(symbol,day.strftime('%m%d%Y'))
        failed_urls = []
        try:
            links += symbol_news(url)
        except (urllib2.URLError, IOError, httplib.HTTPException) as e:
            print e.args
            print url
            failed_urls.append(url)
            pass
        if len(failed_urls) > 0:
            failed_urls = pd.DataFrame(failed_urls)
            failed_urls['Symbol'] = symbol
            if not os.path.exists('datasets/companies/failed_urls.csv'):
                failed_urls.to_csv('datasets/companies/failed_urls.csv', encoding='utf8')
            else:
                all_failed = pd.read_csv('datasets/companies/failed_urls.csv', index_col=0)
                all_failed = pd.concat([all_failed, failed_urls], axis=1)
                all_failed.to_csv('datasets/companies/failed_urls.csv', encoding='utf8')
    links = pd.DataFrame(links)
    return links

In [4]:
c = s_p_500

In [5]:
articles = []
for symbol in c.Ticker_symbol:
    print symbol
    if not os.path.exists('datasets/companies/{}.csv'.format(symbol)):
        article = get_news_for_symbol(symbol, end_date = date.today())
        article.to_csv('datasets/companies/{}.csv'.format(symbol), encoding='utf8')
        articles.append(article)
    else:
        article = pd.read_csv('datasets/companies/{}.csv'.format(symbol), encoding='utf8', index_col=0)
        articles.append(article)

MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AES
AET
AFL
AMG
A
APD
AKAM
ALK
ALB
AGN
LNT
ALXN
ALLE
ADS
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANTM
AON
APA
AIV
AAPL
AMAT
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AN
AZO
AVB
AVY
BHI
BLL
BAC
BK
BCR
BAX
BBT
BDX
BBBY
BRKB
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BMY
AVGO
BFB
CHRW
CA
COG
CPB
COF
CAH
HSIC
KMX
CCL
CAT
CBG
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
COH
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DLPH
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DG
DLTR
D
DOV
DOW
DPS
DTE
DD
DUK
DNB
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENDP
ETR
EOG
EQT
EFX
EQIX
EQR
ESS
EL
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FISV
FLIR
FLS
FLR
FMC
FTI
FL
F
FTV
FBHS
BEN
FCX
FTR
GPS
GRMN
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HAR
HRS
HIG
HAS
HCA
HCP
HP
HES
HPE
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
ITW
ILMN
IR
INTC
ICE
IBM
IP
I

KeyboardInterrupt: 